# Задание 3: Разведочный анализ

Данные наблюдений за видами (когда и где наблюдается данный вид) типичны для исследований биоразнообразия. 
Крупные международные инициативы поддерживают сбор этих данных волонтерами, например, iNaturalist. 
Благодаря таким инициативам, как GBIF, многие из этих данных также находятся в открытом доступе.

Вы решили поделиться данными полевой кампании, но набор данных все еще требует некоторой очистки и стандартизации. 
Например, координаты могут называться x/y, decimalLatitude/decimalLongitude, lat/long ... 
К счастью, вы знаете о международном стандарте открытых данных для описания данных о событиях/наблюдениях, т.е. Darwin Core (DwC). 
Вместо того, чтобы изобретать собственную модель данных, вы решаете соответствовать этому международному стандарту. 
Последнее улучшит коммуникацию, а также сделает ваши данные совместимыми с GBIF.

Короче говоря, DwC описывает плоскую таблицу (CSV) с согласованным соглашением об именах для имен заголовков и соглашениями о том, как должны быть представлены определенные типы данных (для справки, подробное описание дано здесь). 
В этом руководстве мы сосредоточимся на нескольких существующих терминах, чтобы изучить некоторые элементы очистки данных:

- eventDate : формат дат ISO 6801
- ScientificName : общепринятое научное название вида.
- decimalLatitude / decimalLongitude : координаты вхождения в формате WGS84
- sex: либо мужской , либо женский , чтобы охарактеризовать пол события
- instanceID : идентификатор в наборе данных для идентификации отдельных записей.
- datasetName : статическая строка, определяющая источник данных

Кроме того, дополнительная информация о таксономии будет добавлена с использованием внешнего API-сервиса.


In [20]:
import csv
from datetime import date, timedelta, datetime
from calendar import month_abbr, day_abbr

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [21]:
#spec_filename = "..\\..\\data\\003\\species.csv"
#spec = pd.read_csv(spec_filename, sep=";")
#spec.head()

In [22]:
survey_filename = "..\\..\\data\\003\\surveys.csv"
survey_data = pd.read_csv(survey_filename, sep=",")
#survey_data.head()

In [23]:
'''
Упражнение 1
Сколько уникальных записей содержит набор данных?
'''
#survey_data.count()
survey_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35549 entries, 0 to 35548
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   record_id  35549 non-null  int64  
 1   month      35549 non-null  int64  
 2   day        35549 non-null  int64  
 3   year       35549 non-null  int64  
 4   plot       35549 non-null  int64  
 5   species    33534 non-null  object 
 6   sex_char   33042 non-null  object 
 7   wgt        32283 non-null  float64
dtypes: float64(1), int64(5), object(2)
memory usage: 2.2+ MB


In [24]:
'''
УПРАЖНЕНИЕ 2
Добавьте новый столбец datasetName в набор данных опроса с datasetname в качестве значения для всех записей (статическое значение для всего набора данных).
'''
survey_data["datasetname"] = "Ecological Archives E090-118-D1."

survey_data.head()

,record_id,month,day,year,plot,species,sex_char,wgt,datasetname
0,1,7,16,1977,2,NaN,M,NaN,Ecological Archives E090-118-D1.
1,2,7,16,1977,3,NaN,M,NaN,Ecological Archives E090-118-D1.
2,3,7,16,1977,2,DM,F,NaN,Ecological Archives E090-118-D1.
3,4,7,16,1977,7,DM,M,NaN,Ecological Archives E090-118-D1.
4,5,7,16,1977,3,DM,M,NaN,Ecological Archives E090-118-D1.


In [25]:
'''
УПРАЖНЕНИЕ 3
Получите список уникальных значений для столбца sex_char.
'''

survey_data["sex_char"].unique()

array(['M', 'F', nan, 'R', 'P', 'Z'], dtype=object)

In [26]:
survey_data = survey_data.rename(columns={'sex_char': 'verbatimSex'})
survey_data.head()


,record_id,month,day,year,plot,species,verbatimSex,wgt,datasetname
0,1,7,16,1977,2,NaN,M,NaN,Ecological Archives E090-118-D1.
1,2,7,16,1977,3,NaN,M,NaN,Ecological Archives E090-118-D1.
2,3,7,16,1977,2,DM,F,NaN,Ecological Archives E090-118-D1.
3,4,7,16,1977,7,DM,M,NaN,Ecological Archives E090-118-D1.
4,5,7,16,1977,3,DM,M,NaN,Ecological Archives E090-118-D1.


In [30]:
'''
УПРАЖНЕНИЕ 4
Выразите отображение значений (например, M -> male) в объект словаря Python с именем переменной sex_dict. Значения Z соответствуют Not a Number, который можно определить как np.nan.
Используйте словарь sex_dict, чтобы заменить значения в столбце verbatimSex новыми значениями и сохранить сопоставленные значения в новом столбце «пол» кадра данных.
'''

sex_dict = dict({'M': 'male', 'F': 'female', 'nan': np.nan, 'Z': np.nan, 'R' : 'male', 'P': 'female'})
survey_data["sex"] = survey_data["verbatimSex"].map(sex_dict)

survey_data["sex"].unique()

array(['male', 'female', nan], dtype=object)

In [27]:
'''
УПРАЖНЕНИЕ 5
Создайте горизонтальную гистограмму, сравнивая количество записей о мужчинах, женщинах и неизвестных (NaN) в наборе данных.
'''

